In [ ]:
# Instalando as bibliotecas necessárias (execute apenas se ainda não estiverem instaladas)
!pip install osmnx networkx folium geopy shapely scikit-learn geopandas geojson pyproj heapq

In [1]:
import json
import os
import osmnx as ox
import networkx as nx
import folium
from folium.plugins import PolyLineTextPath
from geopy.geocoders import Nominatim
from shapely.geometry import Point, LineString
import time
import warnings
import sys
import geopandas as gpd
import geojson
from pyproj import Transformer
import heapq

# Ignorar FutureWarnings temporariamente
warnings.filterwarnings("ignore", category=FutureWarning)

def geocode_address(address):
    """
    Converte um endereço em coordenadas geográficas (latitude e longitude).
    Se múltiplos resultados forem encontrados, permite que o usuário escolha um.
    """
    geolocator = Nominatim(user_agent="shortest_path_tester")
    locations = geolocator.geocode(address, exactly_one=False, limit=5)
    if not locations:
        print("Endereço não encontrado. Tente novamente.")
        return None
    elif len(locations) == 1:
        location = locations[0]
        return (location.latitude, location.longitude, location.address)
    else:
        print("Vários endereços encontrados:")
        for idx, loc in enumerate(locations):
            print(f"{idx + 1}: {loc.address}")
        while True:
            try:
                choice = int(input("Escolha o número do endereço correto (ou 0 para sair): "))
                if choice == 0:
                    return None
                elif 1 <= choice <= len(locations):
                    location = locations[choice - 1]
                    return (location.latitude, location.longitude, location.address)
                else:
                    print("Opção inválida. Tente novamente.")
            except ValueError:
                print("Entrada inválida. Por favor, insira um número.")

def save_preferences(preferences, filename='user_preferences.json'):
    """
    Salva as preferências do usuário em um arquivo JSON.
    """
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(preferences, f, ensure_ascii=False, indent=4)

def load_preferences(filename='user_preferences.json'):
    """
    Carrega as preferências do usuário de um arquivo JSON, se existir.
    Retorna um dicionário de preferências ou None se o arquivo não existir.
    """
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            preferences = json.load(f)
        return preferences
    else:
        return None

def get_user_input(preferences):
    """
    Coleta informações do usuário: endereço de origem, raio de busca, tipo de estabelecimento.
    Usa as preferências carregadas se disponíveis.
    Atualiza o dicionário de preferências com os novos valores.
    """
    # Endereço de origem
    if 'address' in preferences and preferences['address']:
        use_previous = input(f"Deseja usar o endereço salvo '{preferences['address']}'? (s/n): ").lower()
        if use_previous == 's':
            address = preferences['address']
            print(f"Usando endereço anterior: {address}")
        else:
            address = input("Digite o novo endereço de origem: ")
            if address.strip() == '':
                print("Nenhum endereço fornecido. Encerrando a aplicação.")
                return None, None, None
            preferences['address'] = address  # Atualizar preferências
    else:
        address = input("Digite o endereço de origem: ")
        if address.strip() == '':
            print("Nenhum endereço fornecido. Encerrando a aplicação.")
            return None, None, None
        preferences['address'] = address  # Atualizar preferências

    # Raio de busca
    if 'radius' in preferences and preferences['radius']:
        radius_default = preferences['radius']
    else:
        radius_default = 1000  # 1 km como padrão

    radius_input = input(f"Digite o raio de busca em metros (padrão {radius_default}m): ")
    if radius_input.strip() == '':
        radius = radius_default  # Usar raio anterior ou padrão
    else:
        try:
            radius = int(radius_input)
            preferences['radius'] = radius  # Atualizar preferências
        except ValueError:
            print("Entrada inválida para o raio. Usando o valor padrão.")
            radius = radius_default

    # Tipo de estabelecimento (cuisine)
    if 'cuisine' in preferences and preferences['cuisine']:
        cuisine_default = preferences['cuisine']
    else:
        cuisine_default = 'pizza'

    cuisine_input = input(f"Digite o tipo de estabelecimento para buscar (padrão '{cuisine_default}'): ")
    if cuisine_input.strip() == '':
        cuisine = cuisine_default  # Usar tipo anterior ou padrão
    else:
        cuisine = cuisine_input
        preferences['cuisine'] = cuisine  # Atualizar preferências

    return address, radius, cuisine

import time

# Função para gerar uma grade de pontos ao redor do ponto de origem
def generate_grid_points(origin_point, total_radius, sub_radius):
    import numpy as np
    from geopy.distance import geodesic

    lat, lon = origin_point
    # Calcular o deslocamento em graus para o sub_raio
    delta = sub_radius / 111320  # Aproximadamente 111.320 metros por grau
    grid_points = []
    steps = int(total_radius / sub_radius)
    for i in range(-steps, steps + 1):
        for j in range(-steps, steps + 1):
            new_lat = lat + i * delta
            new_lon = lon + j * delta
            if geodesic((lat, lon), (new_lat, new_lon)).meters <= total_radius:
                grid_points.append((new_lat, new_lon))
    return grid_points

def get_pois(G_projected, origin_point_geo, radius=5000, cuisine='pizza'):
    """
    Busca pontos de interesse (POIs) próximos à origem com base no tipo de estabelecimento.
    Permite busca por substring na tag 'cuisine'.
    """
    tags = {'amenity': 'restaurant'}
    pois = ox.geometries_from_point(origin_point_geo, tags=tags, dist=radius)
    
    if pois.empty:
        print("Nenhum estabelecimento encontrado na área.")
        return [], [], []
    else:
        print(f"{len(pois)} estabelecimentos encontrados.")
        
        # Filtrar por substring na coluna 'cuisine'
        if 'cuisine' in pois.columns:
            
            pois = pois[pois['cuisine'].str.contains(cuisine, case=False, na=False)]
            print(f"{len(pois)} estabelecimentos correspondem à busca por '{cuisine}'.")
        else:
            print("A coluna 'cuisine' não está presente nos dados.")
            pois = pois[pois['name'].str.contains(cuisine, case=False, na=False)]
            print(f"{len(pois)} estabelecimentos correspondem à busca por nome contendo '{cuisine}'.")
        
        if pois.empty:
            print("Nenhum estabelecimento correspondente encontrado após filtragem.")
            return [], [], []
        
        # Reprojetar para o CRS do grafo projetado
        pois_projected = pois.to_crs(G_projected.graph['crs'])
        
        # Calcular os centróides
        pois_centroids_projected = pois_projected.geometry.centroid
        
        # Obter coordenadas projetadas
        pois_coords_proj = [(point.x, point.y) for point in pois_centroids_projected]
        
        # Encontrar os nós mais próximos
        pois_nodes = ox.distance.nearest_nodes(
            G_projected, 
            X=[coord[0] for coord in pois_coords_proj], 
            Y=[coord[1] for coord in pois_coords_proj]
        )
        
        # Obter nomes dos estabelecimentos
        pois_names = pois['name'].tolist()
        
        # Converter para coordenadas geográficas para plotagem
        pois_centroids_geo = pois_centroids_projected.to_crs(epsg=4326)
        pois_coords_geo = [(point.y, point.x) for point in pois_centroids_geo]
        
        return pois_nodes, pois_coords_geo, pois_names

def select_closest_destinations(G_projected, origin_node, destination_nodes, destination_names, num_destinations=None):
    """
    Ordena os destinos com base na distância ao nó de origem e seleciona os N mais próximos.
    """
    # Calcular distâncias do nó de origem para todos os destinos
    distances = []
    for idx, dest_node in enumerate(destination_nodes):
        try:
            length = nx.shortest_path_length(G_projected, origin_node, dest_node, weight='length')
            distances.append((length, dest_node, destination_names[idx]))
        except nx.NetworkXNoPath:
            continue  # Ignorar destinos sem caminho disponível
    
    # Ordenar por distância
    distances.sort(key=lambda x: x[0])
    
    if not distances:
        print("Nenhum destino alcançável encontrado.")
        return [], []
    
    # Perguntar ao usuário quantos destinos deseja
    if num_destinations is None:
        num_destinations_input = input(f"Total de destinos encontrados: {len(distances)}. Quantos deseja traçar? (pressione Enter para todos): ")
        global previous_num_destinations
        if num_destinations_input == '':
            num_destinations = len(distances)
            previous_num_destinations = num_destinations  # Persistir para uso futuro
        else:
            num_destinations = int(num_destinations_input)
            previous_num_destinations = num_destinations  # Persistir para uso futuro
    else:
        num_destinations = min(num_destinations, len(distances))
    
    # Selecionar os N destinos mais próximos
    selected_destinations = distances[:num_destinations]
    
    # # Perguntar se o usuário deseja ver os nomes dos destinos
    # show_names = input("Deseja ver os nomes dos destinos selecionados? (s/n): ").lower()
    # if show_names == 's':
    #     for idx, (dist, node, name) in enumerate(selected_destinations):
    #         print(f"{idx + 1}: {name} (Distância: {dist:.2f} metros)")
    
    # Extrair os nós e nomes selecionados
    selected_nodes = [node for dist, node, name in selected_destinations]
    selected_names = [name for dist, node, name in selected_destinations]
    selected_dists = [dist for dist, node, name in selected_destinations]
    
    return selected_nodes, selected_names, selected_dists


def bidirectional_a_star(G, source, target, heuristic):
    """
    Implementa o algoritmo Bidirectional A* para encontrar o caminho mais curto entre source e target.

    Parâmetros:
    - G: networkx.DiGraph - Grafo direcionado.
    - source: int - Nó de origem.
    - target: int - Nó de destino.
    - heuristic: função - Função heurística admissível que estima a distância entre dois nós.

    Retorna:
    - full_path: lista - Lista de nós representando o caminho mais curto.
    """
    # Inicialização das buscas em ambas as direções
    forward_queue = []
    backward_queue = []
    heapq.heappush(forward_queue, (heuristic(source, target), 0, source))
    heapq.heappush(backward_queue, (heuristic(target, source), 0, target))

    forward_visited = {source: 0}
    backward_visited = {target: 0}

    forward_parents = {source: None}
    backward_parents = {target: None}

    meeting_node = None
    best_cost = float('inf')

    while forward_queue and backward_queue:
        # Verifica a condição de parada
        min_forward_priority = forward_queue[0][0]
        min_backward_priority = backward_queue[0][0]
        if best_cost <= min_forward_priority + min_backward_priority:
            break

        # Expansão na direção forward
        if forward_queue:
            current_forward_priority, current_forward_cost, current_forward_node = heapq.heappop(forward_queue)
            if current_forward_node in backward_visited:
                total_cost = current_forward_cost + backward_visited[current_forward_node]
                if total_cost < best_cost:
                    best_cost = total_cost
                    meeting_node = current_forward_node
            for neighbor in G.successors(current_forward_node):
                edge_data = G.get_edge_data(current_forward_node, neighbor, default={})
                length = edge_data.get('length', 1)
                cost = forward_visited[current_forward_node] + length
                if neighbor not in forward_visited or cost < forward_visited[neighbor]:
                    forward_visited[neighbor] = cost
                    priority = cost + heuristic(neighbor, target)
                    heapq.heappush(forward_queue, (priority, cost, neighbor))
                    forward_parents[neighbor] = current_forward_node

        # Expansão na direção backward
        if backward_queue:
            current_backward_priority, current_backward_cost, current_backward_node = heapq.heappop(backward_queue)
            if current_backward_node in forward_visited:
                total_cost = current_backward_cost + forward_visited[current_backward_node]
                if total_cost < best_cost:
                    best_cost = total_cost
                    meeting_node = current_backward_node
            for neighbor in G.predecessors(current_backward_node):
                edge_data = G.get_edge_data(neighbor, current_backward_node, default={})
                length = edge_data.get('length', 1)
                cost = backward_visited[current_backward_node] + length
                if neighbor not in backward_visited or cost < backward_visited[neighbor]:
                    backward_visited[neighbor] = cost
                    priority = cost + heuristic(neighbor, source)
                    heapq.heappush(backward_queue, (priority, cost, neighbor))
                    backward_parents[neighbor] = current_backward_node

    if meeting_node is None:
        raise nx.NetworkXNoPath(f"Nenhuma rota encontrada entre {source} e {target} usando Bidirectional A*.")

    # Reconstrução do caminho
    path_forward = []
    node = meeting_node
    while node is not None:
        path_forward.append(node)
        node = forward_parents[node]
    path_forward.reverse()

    path_backward = []
    node = backward_parents[meeting_node]
    while node is not None:
        path_backward.append(node)
        node = backward_parents[node]

    full_path = path_forward + path_backward
    return full_path

def calculate_routes(G, origin_node, target_nodes, algorithms=['dijkstra', 'astar', 'bellman_ford', 'bidirectional_dijkstra', 'bidirectional_a_star']):
    """
    Calcula as rotas mais curtas do nó de origem para cada nó de destino usando os algoritmos especificados.
    
    Parâmetros:
    - G: networkx.DiGraph - Grafo direcionado.
    - origin_node: int - Nó de origem.
    - target_nodes: lista - Lista de nós de destino.
    - algorithms: lista - Lista de algoritmos a serem utilizados.
    
    Retorna:
    - routes: dict - Dicionário contendo as rotas para cada algoritmo.
    - avg_times: dict - Dicionário contendo o tempo médio de execução para cada algoritmo.
    """
    routes = {alg: [] for alg in algorithms}
    times = {alg: [] for alg in algorithms}
    
    for alg in algorithms:
        for target in target_nodes:
            try:
                start_time = time.time()
                if alg == 'dijkstra':
                    route = nx.shortest_path(G, origin_node, target, weight='length')
                elif alg == 'astar':
                    route = nx.astar_path(
                        G, 
                        origin_node, 
                        target, 
                        weight='length', 
                        heuristic=lambda u, v: ox.distance.euclidean(
                            G.nodes[u]['y'], G.nodes[u]['x'], 
                            G.nodes[v]['y'], G.nodes[v]['x']
                        )
                    )
                elif alg == 'bellman_ford':
                    route = nx.bellman_ford_path(G, origin_node, target, weight='length')
                elif alg == 'bidirectional_dijkstra':
                    path = nx.bidirectional_dijkstra(G, origin_node, target, weight='length')[1]
                    route = path
                elif alg == 'bidirectional_a_star':
                    heuristic = lambda u, v: ox.distance.euclidean(
                        G.nodes[u]['y'], G.nodes[u]['x'], 
                        G.nodes[v]['y'], G.nodes[v]['x']
                    )
                    route = bidirectional_a_star(G, origin_node, target, heuristic)
                else:
                    raise ValueError("Algoritmo não suportado.")
                end_time = time.time()
                routes[alg].append(route)
                times[alg].append(end_time - start_time)
            except nx.NetworkXNoPath:
                print(f"Nenhuma rota encontrada para o nó {target} usando {alg}.")
            except Exception as e:
                print(f"Erro ao calcular rota para o nó {target} usando {alg}: {e}")
    
    avg_times = {alg: (sum(times[alg]) / len(times[alg]) if times[alg] else 0) for alg in algorithms}
    return routes, avg_times

def routes_to_geojson(G, routes, alg='dijkstra'):
    """
    Converte rotas para o formato GeoJSON.
    
    Parâmetros:
    - G: networkx.DiGraph - Grafo direcionado.
    - routes: dict - Dicionário contendo as rotas para cada algoritmo.
    - alg: str - Algoritmo específico a ser convertido para GeoJSON.
    
    Retorna:
    - feature_collection: geojson.FeatureCollection - Coleção de features GeoJSON.
    """
    features = []
    for route in routes[alg]:
        if len(route) < 2:
            continue  # Rotas inválidas com menos de 2 nós
        try:
            line = LineString([(G.nodes[node]['x'], G.nodes[node]['y']) for node in route])
            feature = geojson.Feature(geometry=line, properties={"algorithm": alg})
            features.append(feature)
        except Exception as e:
            print(f"Erro ao criar LineString para {alg}: {e}")
            continue
    feature_collection = geojson.FeatureCollection(features)
    return feature_collection


from folium.plugins import PolyLineTextPath

def plot_routes_subset(G, origin_point_geo, routes, destination_coords_geo, destination_names, destination_dists, transformer, algorithms=['dijkstra', 'astar', 'bellman_ford', 'bidirectional_dijkstra', 'bidirectional_a_star'], limit=10):
    """
    Plota as rotas calculadas em camadas separadas no mapa interativo usando Folium,
    limitando o número de rotas por algoritmo para 'limit', e adiciona setas para indicar o sentido do percurso.

    Parâmetros:
    - G: networkx.DiGraph - Grafo direcionado.
    - origin_point_geo: tuple - Coordenadas geográficas da origem (lat, lon).
    - routes: dict - Dicionário contendo as rotas para cada algoritmo.
    - pizzeria_coords_geo: lista - Lista de coordenadas geográficas das pizzarias.
    - transformer: pyproj.Transformer - Objeto Transformer para conversão de CRS.
    - algorithms: lista - Lista de algoritmos a serem plotados.
    - limit: int - Número máximo de rotas por algoritmo a serem plotadas.

    Retorna:
    - m: folium.Map - Objeto de mapa Folium com as rotas plotadas.
    """
    # Criar mapa centrado na origem
    m = folium.Map(location=origin_point_geo, zoom_start=13)

    # Adicionar marcador para a origem
    folium.Marker(
        location=origin_point_geo,
        popup="Origem",
        icon=folium.Icon(color='blue', icon='home')
    ).add_to(m)

    # Adicionar marcadores para as pizzarias
    for idx, (coord, name, dist) in enumerate(zip(destination_coords_geo, destination_names, destination_dists)):
        folium.Marker(
            location=coord,
            popup=f"Destino {idx+1}: {name} (Distância: {dist:.2f} metros)",
            icon=folium.Icon(color='red', icon='cutlery')
        ).add_to(m)

    # Definir cores para os algoritmos
    color_map = {
        'dijkstra': 'green',
        'astar': 'purple',
        'bellman_ford': 'orange',
        'bidirectional_dijkstra': 'blue',
        'bidirectional_a_star': 'darkgreen'
    }

    # Adicionar rotas em camadas separadas, limitando o número de rotas
    for alg in algorithms:
        layer = folium.FeatureGroup(name=f"Rotas {alg.replace('_', ' ').capitalize()}")
        for route in routes[alg][:limit]:
            try:
                if len(route) < 2:
                    continue  # Rotas inválidas com menos de 2 nós

                # Obter as coordenadas projetadas dos nós da rota
                route_proj = [(G.nodes[node]['x'], G.nodes[node]['y']) for node in route]

                # Converter para geográficas
                route_geo = [transformer.transform(x, y) for x, y in route_proj]

                # Rearranjar para (lat, lon) para Folium
                route_geo_latlon = [(lat, lon) for lon, lat in route_geo]

                # Adicionar a rota como PolyLine
                polyline = folium.PolyLine(
                    route_geo_latlon,
                    color=color_map.get(alg, 'blue'),
                    weight=5,
                    opacity=1,
                    popup=f"Rota {alg.replace('_', ' ').capitalize()}"
                )
                polyline.add_to(layer)

                # Adicionar setas para indicar o sentido do percurso usando PolyLineTextPath
                arrow_symbol = '\u27A4'  # Símbolo Unicode para seta para a direita
                folium.plugins.PolyLineTextPath(
                    polyline,
                    arrow_symbol * 3, 
                    repeat=True,      # Repetir o símbolo para aumentar o tamanho
                    offset=9,         # Ajusta a posição vertical do símbolo
                    attributes={
                        'fill': color_map.get(alg, 'blue'),
                        'font-weight': 'normal',
                        'font-size': '8'  # Aumenta o tamanho da seta
                    }
                ).add_to(layer)
            except Exception as e:
                print(f"Erro ao plotar rota {alg}: {e}")
        layer.add_to(m)

    # Adicionar controle de camadas
    folium.LayerControl().add_to(m)

    return m

# Funções de ALT (Opcional, se desejar implementar posteriormente)
def select_landmarks(G, num_landmarks=5):
    """
    Seleciona 'num_landmarks' nós como landmarks utilizando uma estratégia simples.
    """
    import random
    return random.sample(list(G.nodes()), num_landmarks)

def compute_landmark_distances(G, landmarks):
    """
    Computa e armazena as distâncias de cada landmark para todos os outros nós.
    """
    landmark_distances = {}
    for landmark in landmarks:
        lengths = nx.single_source_dijkstra_path_length(G, landmark, weight='length')
        landmark_distances[landmark] = lengths
    return landmark_distances

def alt_heuristic(G, landmark_distances, landmarks, u, v):
    """
    Heurística ALT para o A* baseada em landmarks.
    """
    heuristics = []
    for landmark in landmarks:
        if u in landmark_distances[landmark] and v in landmark_distances[landmark]:
            heuristics.append(abs(landmark_distances[landmark][u] - landmark_distances[landmark][v]))
    return max(heuristics) if heuristics else 0

if __name__ == "__main__":
    # Carregar preferências se existirem
    preferences = load_preferences()
    if preferences is None:
        preferences = {}

    while True:
        # Coletar informações do usuário
        address, radius, cuisine = get_user_input(preferences)
        if not address:
            print("Nenhum endereço fornecido. Encerrando a aplicação.")
            sys.exit()
    
        # Geocodificar o endereço
        geocode_result = geocode_address(address)
        if geocode_result is None:
            print("O endereço fornecido não pode ser convertido em coordenadas geográficas. Tente novamente!\n")
            continue  # Tentar novamente
        else:
            origin_point = (geocode_result[0], geocode_result[1])
            origin_address = geocode_result[2]
            print(f"Endereço selecionado: {origin_address}")
    
        # Baixar grafo de ruas
        print("Baixando dados de ruas do OSM...")
        try:
            G = ox.graph_from_point(
                origin_point, 
                dist=radius, 
                network_type='drive', 
                simplify=False  # Desativar simplificação para maior densidade
            )
            # Reprojetar o grafo para um CRS projetado (por exemplo, UTM)
            G_projected = ox.project_graph(G)
            print("Grafo de ruas carregado e reprojetado.")
        except Exception as e:
            print(f"Erro ao baixar ou processar o grafo: {e}")
            sys.exit()
    
        # Verificar se o grafo é direcionado
        if not G_projected.is_directed():
            print("O grafo não é direcionado. Certifique-se de que 'network_type'='drive' está sendo usado.")
            sys.exit()
    
        # Definir o transformador
        crs_projected = G_projected.graph['crs']
        transformer = Transformer.from_crs(crs_projected, "epsg:4326", always_xy=True)
    
        # Converter as coordenadas da origem para o CRS projetado
        origin_gdf = gpd.GeoDataFrame(
            {'geometry': [Point(origin_point[1], origin_point[0])]},  # (lon, lat)
            crs='epsg:4326'  # CRS geográfico
        )
        origin_projected = origin_gdf.to_crs(crs_projected)
        origin_x_proj, origin_y_proj = origin_projected.geometry.x[0], origin_projected.geometry.y[0]
    
        # Encontrar nó de origem no grafo projetado
        try:
            origin_node = ox.distance.nearest_nodes(G_projected, X=origin_x_proj, Y=origin_y_proj)
            origin_node_coords = (G_projected.nodes[origin_node]['y'], G_projected.nodes[origin_node]['x'])
            print(f"Nó de Origem: {origin_node}")
            print(f"Coordenadas do Nó de Origem: {origin_node_coords}")
        except Exception as e:
            print(f"Erro ao encontrar o nó de origem: {e}")
            sys.exit()
    
        # Buscar estabelecimentos
        destination_nodes, destination_coords_geo, destination_names = get_pois(
            G_projected, origin_point, radius=radius, cuisine=cuisine
        )
        
        if not destination_nodes:
            print("Nenhum destino encontrado. Tente novamente com outros parâmetros.\n")
            continue  # Reiniciar o loop
    
        # Selecionar os destinos mais próximos
        selected_nodes, selected_names, selected_dists = select_closest_destinations(
            G_projected, origin_node, destination_nodes, destination_names
        )
        if not selected_nodes:
            print("Nenhum destino selecionado. Tente novamente.\n")
            continue  # Reiniciar o loop
        else:
            break
    
    # Novo código: Exibir o número de nós e arestas do grafo
    print(f"O grafo utilizado para o cálculo das rotas possui {G_projected.number_of_nodes()} nós e {G_projected.number_of_edges()} arestas.")

    # Atualizar o endereço completo nas preferências
    preferences['address_full'] = origin_address

    # Salvar preferências atualizadas
    save_preferences(preferences)

    # Filtrar as coordenadas e nomes correspondentes aos destinos selecionados
    selected_coords_geo = []
    for node in selected_nodes:
        idx = destination_nodes.index(node)
        selected_coords_geo.append(destination_coords_geo[idx])

    # Calcular rotas
    algorithms = ['dijkstra', 'astar', 'bellman_ford', 'bidirectional_dijkstra', 'bidirectional_a_star']
    routes, avg_times = calculate_routes(G_projected, origin_node, selected_nodes, algorithms=algorithms)

    # Exibir tempos médios
    for alg, avg_time in avg_times.items():
        print(f"Tempo Médio {alg.replace('_', ' ').capitalize()}: {avg_time:.6f} segundos")

    # Verificar número de rotas
    routes_limit=0
    for alg in algorithms:
        print(f"Total de rotas para {alg}: {len(routes[alg])}")
        if routes_limit < len(routes[alg]):
            routes_limit = len(routes[alg])

    # # Inspecionar as primeiras 3 rotas de cada algoritmo
    # for alg in algorithms:
    #     if routes[alg]:
    #         print(f"\nPrimeiras 3 rotas para {alg.replace('_', ' ').capitalize()}:")
    #         for i, route in enumerate(routes[alg][:3], start=1):
    #             route_coords = [(G_projected.nodes[node]['y'], G_projected.nodes[node]['x']) for node in route]
    #             print(f" Rota {i} ({alg.replace('_', ' ').capitalize()}): {route_coords[:3]} ... {route_coords[-3:]}")
    #     else:
    #         print(f"\nNenhuma rota encontrada para {alg}.")

    # Plotar as rotas usando plot_routes_subset
    limit=routes_limit
    m_subset = plot_routes_subset(G_projected, origin_point, routes, selected_coords_geo, selected_names, selected_dists, transformer, algorithms=algorithms, limit=limit)

    # Salvar e exibir o mapa
    m_subset.save('mapa_rotas.html')
    print("Mapa salvo como 'mapa_rotas.html'. Abra-o no seu navegador para visualização.")

    # Visualizar rotas no Jupyter
    try:
        display(m_subset)
    except ImportError:
        pass  # Se não estiver em um ambiente Jupyter, ignore            


Deseja usar o endereço salvo 'Edifício 5, Rua Alzira Cortes'? (s/n):  s


Usando endereço anterior: Edifício 5, Rua Alzira Cortes


Digite o raio de busca em metros (padrão 5000m):  
Digite o tipo de estabelecimento para buscar (padrão 'pizza'):  


Endereço selecionado: Rua Alzira Cortes, Santa Marta, Botafogo, Rio de Janeiro, Região Geográfica Imediata do Rio de Janeiro, Região Metropolitana do Rio de Janeiro, Região Geográfica Intermediária do Rio de Janeiro, Rio de Janeiro, Região Sudeste, 22260-040, Brasil
Baixando dados de ruas do OSM...
Grafo de ruas carregado e reprojetado.
Nó de Origem: 5303888106
Coordenadas do Nó de Origem: (7461210.251086254, 685625.3177008745)
833 estabelecimentos encontrados.
60 estabelecimentos correspondem à busca por 'pizza'.


Total de destinos encontrados: 59. Quantos deseja traçar? (pressione Enter para todos):  


O grafo utilizado para o cálculo das rotas possui 26454 nós e 39375 arestas.
Tempo Médio Dijkstra: 0.018709 segundos
Tempo Médio Astar: 0.005967 segundos
Tempo Médio Bellman ford: 0.307737 segundos
Tempo Médio Bidirectional dijkstra: 0.018118 segundos
Tempo Médio Bidirectional a star: 0.000935 segundos
Total de rotas para dijkstra: 59
Total de rotas para astar: 59
Total de rotas para bellman_ford: 59
Total de rotas para bidirectional_dijkstra: 59
Total de rotas para bidirectional_a_star: 59
Mapa salvo como 'mapa_rotas.html'. Abra-o no seu navegador para visualização.
